In [2]:
import numpy as np
import pandas as pd
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt # data visualization
import ipywidgets as widgets # interactive widgets
from ipywidgets import Box


In [3]:
df = pd.read_csv('../Data/tracks_cleaned.csv')
df.describe()

,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,num_artists,year
count,469281.000000,4.692810e+05,469281.000000,469281.000000,469281.000000,469281.000000,469281.000000,469281.000000,469281.000000,469281.000000,469281.000000,469281.000000,469281.000000,469281.000000,469281.000000,469281.000000,469281.000000
mean,27.578334,2.299850e+05,0.044155,0.563647,0.542439,5.224840,-10.200712,0.659208,0.104881,0.449396,0.113413,0.213992,0.552442,118.465459,3.873500,1.295914,1988.594946
std,18.366320,1.270919e+05,0.205439,0.166195,0.251780,3.517928,5.086349,0.473976,0.179904,0.348656,0.266952,0.184386,0.257641,29.784032,0.472858,0.887235,22.813764
min,0.000000,3.344000e+03,0.000000,0.000000,0.000000,0.000000,-60.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1922.000000
25%,13.000000,1.750530e+05,0.000000,0.453000,0.344000,2.000000,-12.887000,0.000000,0.034000,0.096900,0.000000,0.098300,0.346000,95.549000,4.000000,1.000000,1974.000000
50%,27.000000,2.149070e+05,0.000000,0.577000,0.550000,5.000000,-9.233000,1.000000,0.044300,0.422000,0.000024,0.139000,0.564000,117.363000,4.000000,1.000000,1992.000000
75%,41.000000,2.638000e+05,0.000000,0.686000,0.749000,8.000000,-6.482000,1.000000,0.076400,0.784000,0.009460,0.278000,0.769000,136.335000,4.000000,1.000000,2007.000000
max,98.000000,5.621218e+06,1.000000,0.991000,1.000000,11.000000,5.376000,1.000000,0.971000,0.996000,1.000000,1.000000,1.000000,243.507000,5.000000,58.000000,2021.000000


## Using all features

Since Key is already an estimated value instead of actual we will drop it from the frame. Additonally, we drop release_date due to the inconsisitencies in the data where release_months are only available for some of the data. Since we have already created a column called year in the data cleaning part, we are still able to use that and take advantage of the strong relation between year and popularity

In [4]:
df.drop(['key', 'release_date'],axis=1, inplace=True)

time_signature_df=pd.get_dummies(df["time_signature"])
df = pd.concat([df,time_signature_df],axis=1)
df['mode'] = np.where(df['mode']=='Major', 1, 0)

 Modeling the data + Sets Prep

In [5]:
X= df.loc[:,df.columns !="popularity"] # all the features except popularity
y = df["popularity"] 

In [6]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 469281 entries, 0 to 469280
Data columns (total 20 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   duration_ms       469281 non-null  int64  
 1   explicit          469281 non-null  int64  
 2   danceability      469281 non-null  float64
 3   energy            469281 non-null  float64
 4   loudness          469281 non-null  float64
 5   mode              469281 non-null  int64  
 6   speechiness       469281 non-null  float64
 7   acousticness      469281 non-null  float64
 8   instrumentalness  469281 non-null  float64
 9   liveness          469281 non-null  float64
 10  valence           469281 non-null  float64
 11  tempo             469281 non-null  float64
 12  time_signature    469281 non-null  int64  
 13  num_artists       469281 non-null  int64  
 14  year              469281 non-null  int64  
 15  0                 469281 non-null  uint8  
 16  1                 46

In [11]:
train_scores = []
test_scores = []

for i in range(1):
    # separate the data to training and testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    # save as np.array
    X_train = np.array(X_train)
    X_test = np.array(X_test)
    y_train = np.array(y_train) 
    y_test = np.array(y_test)

    #creating a regression model
    model_decision_tree = DecisionTreeRegressor()
    model_decision_tree.fit(X_train, y_train)

    %store model_decision_tree

    #We estimate models’ success rate in predicting the songs using the score() function which calculates the coefficient of determination. A score which is closer to 1 means the regressor is more accurate.
    train = model_decision_tree.score(X_train, y_train)
    test = model_decision_tree.score(X_test, y_test)

    train_scores.append(train)
    test_scores.append(test)

# Calculate average train and test accuracy
avg_train = sum(train_scores) / len(train_scores)
avg_test = sum(test_scores) / len(test_scores)

print("Average Train Accuracy = " + str(avg_train))
print("Average Test Accuracy = " + str(avg_test))

Stored 'model_decision_tree' (DecisionTreeRegressor)
Average Train Accuracy = 0.9950156144813767
Average Test Accuracy = 0.02713350038355622


As such, when non-neglgible factors are accounted for, the success rate of a decision-tree model in predicting popularity in test set is 0.02

## Dropping Non-essential factors

In [16]:
df = pd.read_csv('../Data/tracks_cleaned.csv')
df.describe()

df.drop(['key', 'release_date', 'num_artists', 'mode', 'speechiness', 'liveness', 'valence', 'tempo', 'time_signature'],axis=1, inplace=True)


In [17]:
X= df.loc[:,df.columns !="popularity"] # all the features except popularity
y = df["popularity"] 

In [18]:
train_scores = []
test_scores = []

for i in range(1):
    # separate the data to training and testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    # save as np.array
    X_train = np.array(X_train)
    X_test = np.array(X_test)
    y_train = np.array(y_train) 
    y_test = np.array(y_test)

    #creating a regression model
    model_decision_tree = DecisionTreeRegressor()
    model_decision_tree.fit(X_train, y_train)

    #We estimate models’ success rate in predicting the songs using the score() function which calculates the coefficient of determination. A score which is closer to 1 means the regressor is more accurate.
    train = model_decision_tree.score(X_train, y_train)
    test = model_decision_tree.score(X_test, y_test)

    train_scores.append(train)
    test_scores.append(test)

# Calculate average train and test accuracy
avg_train = sum(train_scores) / len(train_scores)
avg_test = sum(test_scores) / len(test_scores)

print("Average Train Accuracy = " + str(avg_train))
print("Average Test Accuracy = " + str(avg_test))

Average Train Accuracy = 0.9948128314723499
Average Test Accuracy = 0.00587793496739597


As seen above, dropping the non-essential features reduces the accuracy by around 0.06

## Using top 6 features from EDA

In [19]:
df = pd.read_csv('../Data/tracks_cleaned.csv')
df.describe()

df.drop(['key', 'release_date', 'num_artists', 'mode', 'speechiness', 'liveness', 'valence', 'tempo', 'time_signature', 'instrumentalness'],axis=1, inplace=True)

In [20]:
X= df.loc[:,df.columns !="popularity"] # all the features except popularity
y = df["popularity"] 

In [21]:
train_scores = []
test_scores = []

for i in range(1):
    # separate the data to training and testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    # save as np.array
    X_train = np.array(X_train)
    X_test = np.array(X_test)
    y_train = np.array(y_train) 
    y_test = np.array(y_test)

    #creating a regression model
    model_decision_tree = DecisionTreeRegressor()
    model_decision_tree.fit(X_train, y_train)

    #We estimate models’ success rate in predicting the songs using the score() function which calculates the coefficient of determination. A score which is closer to 1 means the regressor is more accurate.
    train = model_decision_tree.score(X_train, y_train)
    test = model_decision_tree.score(X_test, y_test)

    train_scores.append(train)
    test_scores.append(test)

# Calculate average train and test accuracy
avg_train = sum(train_scores) / len(train_scores)
avg_test = sum(test_scores) / len(test_scores)

print("Average Train Accuracy = " + str(avg_train))
print("Average Test Accuracy = " + str(avg_test))

Average Train Accuracy = 0.9945074035424195
Average Test Accuracy = -0.015102234350520938


Reducing the number of features further to important features reduces the success score of the decision-tree model further.

## Conclusion

In conclusion, when it comes to decision-tree models, using such a model for prediciton gives really poor scores.

Additionally, addressing the decision tree model, the negative score implies that this model’s performance is worse than just guessing the expected value no matter what song is being tested.